In [1]:
@file:DependsOn("io.ktor:ktor-client-core:3.1.3")
@file:DependsOn("io.ktor:ktor-client-cio:3.1.3")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:3.1.3")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:3.1.3")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-18.1")
user

UserTO(id=d5cbd0c7-793c-4ec3-ac79-93c65d7100ce, username=Johann-18.1)

In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"
val IMPORT_CONFIGURATION_YAML_FILE = "../../data/provision/import-configuration.yaml"

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()


In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking
import com.charleskorn.kaml.Yaml
import kotlinx.serialization.builtins.ListSerializer

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = File(ACCOUNTS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), it) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=c044501a-4558-45ec-bf44-52eaeba83b18, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=06654813-9565-4c49-ad7d-45c240eadebb, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=24180b81-ae28-4b20-a57a-d5a9e8f9518a, name=ING RON, unit=Currency(value=RON))
AccountTO(id=9c00242a-6e6b-44a2-8faa-913fadce7fb0, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=e173a7ea-2fe5-4e61-99d7-d851bc7b64a2, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=306f9b41-359f-44c4-b1a5-ff47b3785aea, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=eed28cc5-75a3-4ee6-8622-fbd3b8f6844e, name=Other, unit=Currency(value=RON))
AccountTO(id=95056f69-010c-4e49-9a1f-354230970fe9, name=BT RON, unit=Currency(value=RON))
AccountTO(id=88716142-50bc-44c7-8438-498e4e4e5c3b, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = File(FUNDS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), it) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=199cd188-7eda-4223-a669-4d0d9f204fba, name=Expenses)
FundTO(id=e9418fdc-b08e-4dbe-b965-fe471cd9766a, name=Work Income)
FundTO(id=5e05647c-f165-4814-bcc9-f3be352a578b, name=Savings)
FundTO(id=48fe02c2-3d46-4514-b45b-763d0cbe2419, name=Gifts)

In [5]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawYaml: String = File(IMPORT_CONFIGURATION_YAML_FILE).readText()
val importConfiguration = Yaml.default.decodeFromString(ImportConfigurationTO.serializer(), dataConfigurationRawYaml)

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=7827c801-0111-473f-8a32-256f1290474f, status=COMPLETED, reason=null)